In [76]:
import sys
import numpy as np
import pandas as pd
from helpers import *
import matplotlib.pyplot as plt
from plots import cross_validation_visualization
%matplotlib notebook


ModuleNotFoundError: No module named 'plots'

In [10]:
filename_measures = 'data/IMPROVE_2015_measures_cs433.csv'
filename_spectra = 'data/IMPROVE_2015_raw_spectra_cs433.csv'
filename_tts = 'data/IMPROVE_2015_train_test_split_cs433.csv'
# filename_sec_deriv = 'data/IMPROVE_2015_2nd-derivative_spectra_cs433.csv'

df_spectra_raw = pd.read_csv(filename_spectra)
df_measures_raw = pd.read_csv(filename_measures)
df_train_test_split_raw = pd.read_csv(filename_tts)
# df_second_derivative = pd.read_csv(filename_sec_deriv, index_col=0)

In [7]:
filename_measures = 'IMPROVE_2015_measures_cs433.csv'
df_meas = pd.read_csv(filename_measures)
dustValues = df_meas['DUSTf:Value']
dustValues = dustValues[~(np.isnan(dustValues))]

### Load data

In [11]:
df_spectra = df_spectra_raw.T
# df_spectra.columns = df_spectra.loc['wavenumber',:]
df_spectra.columns = pd.Float64Index(df_spectra.loc['wavenumber',:], name="")
df_spectra = df_spectra.drop('wavenumber')
df_spectra

,3998.42320552,3997.1375389,3995.85187227,3994.56620564,3993.28053902,3991.99487239,3990.70920577,3989.42353914,3988.13787252,3986.85220589,...,431.983986191,430.698319565,429.41265294,428.126986314,426.841319688,425.555653063,424.269986437,422.984319812,421.698653186,420.412986561
ACAD1_01_06_2015_NM_0_csv,0.35778,0.35760,0.35743,0.35725,0.35708,0.35689,0.35671,0.35652,0.35634,0.35615,...,-0.074765,-0.074292,-0.073728,-0.073047,-0.072208,-0.071185,-0.069949,-0.068499,-0.066864,-0.065079
ACAD1_01_09_2015_NM_0_csv,0.58858,0.58832,0.58806,0.58781,0.58756,0.58731,0.58707,0.58682,0.58657,0.58632,...,-0.099660,-0.099046,-0.098279,-0.097314,-0.096101,-0.094638,-0.092941,-0.091018,-0.088871,-0.086477
ACAD1_01_12_2015_NM_0_csv,0.55091,0.55055,0.55019,0.54984,0.54948,0.54912,0.54875,0.54838,0.54801,0.54764,...,-0.095560,-0.095277,-0.094907,-0.094408,-0.093729,-0.092851,-0.091786,-0.090555,-0.089180,-0.087647
ACAD1_01_15_2015_NM_0_csv,0.49739,0.49715,0.49690,0.49665,0.49640,0.49615,0.49590,0.49564,0.49539,0.49515,...,-0.105530,-0.105590,-0.105570,-0.105420,-0.105160,-0.104810,-0.104370,-0.103790,-0.103030,-0.102080
ACAD1_01_18_2015_NM_0_csv,0.46597,0.46570,0.46543,0.46516,0.46489,0.46461,0.46433,0.46406,0.46379,0.46352,...,-0.098993,-0.098985,-0.098909,-0.098746,-0.098489,-0.098140,-0.097691,-0.097119,-0.096395,-0.095512
ACAD1_01_21_2015_NM_0_csv,0.42318,0.42296,0.42273,0.42250,0.42227,0.42204,0.42181,0.42158,0.42135,0.42111,...,-0.093835,-0.093825,-0.093752,-0.093608,-0.093394,-0.093108,-0.092722,-0.092203,-0.091532,-0.090708
ACAD1_01_24_2015_NM_0_csv,0.41571,0.41546,0.41520,0.41493,0.41466,0.41439,0.41411,0.41384,0.41357,0.41330,...,-0.097176,-0.097236,-0.097219,-0.097098,-0.096879,-0.096589,-0.096227,-0.095764,-0.095158,-0.094399
ACAD1_01_27_2015_NM_0_csv,0.34475,0.34452,0.34430,0.34407,0.34384,0.34361,0.34338,0.34316,0.34293,0.34270,...,-0.082713,-0.082733,-0.082687,-0.082551,-0.082329,-0.082038,-0.081675,-0.081217,-0.080640,-0.079938
ACAD1_01_30_2015_NM_0_csv,0.62313,0.62289,0.62265,0.62240,0.62215,0.62190,0.62164,0.62139,0.62114,0.62090,...,-0.122610,-0.122660,-0.122600,-0.122410,-0.122100,-0.121680,-0.121150,-0.120480,-0.119610,-0.118530
ACAD1_02_02_2015_NM_0_csv,0.43065,0.43043,0.43020,0.42997,0.42974,0.42952,0.42929,0.42906,0.42883,0.42860,...,-0.097128,-0.097142,-0.097079,-0.096926,-0.096684,-0.096367,-0.095968,-0.095459,-0.094799,-0.093968


In [12]:
meta_cols = ['SiteCode','Date','flag','Latitude','Longitude','DUSTf:Unc']
y_col = ['DUSTf:Value']
df_measures = df_measures_raw.set_index('site')
df_measures = df_measures[meta_cols + y_col]
df_measures.index = pd.Index(df_measures.index, name="")
merged = pd.merge(df_spectra, df_measures, left_index=True, right_index=True)

## remove nan

In [13]:
nan_indices = merged['DUSTf:Value'].index[merged['DUSTf:Value'].apply(np.isnan)]
merged.drop(nan_indices, inplace=True)


## X,y creation

In [14]:
X = merged.loc[:, [x for x in merged.columns if x not in y_col and x not in meta_cols]]
y = merged[y_col]

# Cross validation

###### build_k_indices will help to divide the X into 10 partitions, then shuffle it.

In [98]:
def build_k_indices(num_row,k_fold, seed):
    """build k indices for k-fold."""
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

##### compute mse

In [100]:
def compute_mse(y, tx, w):
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.dot(e) / (2 * len(e))
    return mse


#####  ridge regression

In [101]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

#### build polynomial

In [102]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

#### cross validation

In [167]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train
    te_indice = k_indices[k] ## Choose one partition as test set
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    
    y_te = y[np.array(te_indice)]
    y_tr = y[np.array(tr_indice)]
    x_te = x.iloc[te_indice]
    x_tr = x.iloc[tr_indice]
    # form data with polynomial degree
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)
    # ridge regression
    w = ridge_regression(y_tr, tx_tr, lambda_)
    # calculate the loss for train and test data
    loss_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, w))
    loss_te = np.sqrt(2 * compute_mse(y_te, tx_te, w))
    return loss_tr, loss_te,w

###### demo

##### cross_validation_visualization

In [168]:
def cross_validation_visualization(lambds, mse_tr, mse_te):
    """visualization the curves of mse_tr and mse_te."""
    plt.semilogx(lambds, mse_tr, marker=".", color='b', label='train error')
    plt.semilogx(lambds, mse_te, marker=".", color='r', label='test error')
    plt.xlabel("lambda")
    plt.ylabel("rmse")
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    plt.savefig("cross_validation")

In [169]:
def cross_validation_demo():
    seed = 12
    degree = 1
    k_fold = 10
    lambdas = np.logspace(-4, 0, 30)
    # split data in k fold
    num_row = y.shape[0]
    k_indices = build_k_indices(num_row, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # cross validation
    for lambda_ in lambdas:
        rmse_tr_tmp = []
        rmse_te_tmp = []
        for k in range(k_fold):
            loss_tr, loss_te,_ = cross_validation(y, X, k_indices, k, lambda_, degree)
            rmse_tr_tmp.append(loss_tr)
            rmse_te_tmp.append(loss_te)
        rmse_tr.append(np.mean(rmse_tr_tmp))
        rmse_te.append(np.mean(rmse_te_tmp))

    cross_validation_visualization(lambdas, rmse_tr, rmse_te)

cross_validation_demo()

[array([19, 13]), array([9, 7]), array([14, 16]), array([10,  8]), array([5, 4]), array([ 1, 18]), array([ 0, 12]), array([15,  3]), array([ 2, 17]), array([ 6, 11])]
